In [3]:
import sys, os

sys.path.append("/home/yizhou7/mariposa/src")
os.chdir("/home/yizhou7/mariposa/")

from enum import Enum
from collections import Counter

from debugger.demo_utils import Report
from debugger.debugger import Debugger, DbgMode
from debugger.bench_viewer import BenchViewer
from debugger.mutant_info import TraceFailure
from utils.analysis_utils import Categorizer, fmt_percent
from benchmark_consts import *

In [4]:
revs = BenchViewer(FIXED_POINT_TARGETS, DbgMode.SINGLETON)

In [ ]:
def fixable_by_erase(r: Report):
    assert len(r.stabilized) != 0
    return "erase" in r.stabilized.action.values

def fixable_by_inst(r: Report):
    assert len(r.stabilized) != 0
    return "inst_keep" in r.stabilized.action.values or "inst_replace" in r.stabilized.action.values

to_fixes = Counter()
uk_fixes = Counter()

for k, rev in revs.items():
    r = rev.build_report()

    fixes = []

    if fixable_by_erase(r):
        fixes.append("erase")
    if fixable_by_inst(r):
        fixes.append("inst")

    if len(fixes) == 2:
        fixes.append("both")

    if k in f_modes[TraceFailure.FAST_UNKNOWN]:
        uk_fixes.update(fixes)
    elif k in f_modes[TraceFailure.TIMEOUT]:
        to_fixes.update(fixes)

to_count = len(f_modes[TraceFailure.TIMEOUT])

print("Timeouts", to_count)
print("fixed by")
for i in sorted(to_fixes, key=lambda x: to_fixes[x], reverse=True):
    print(f"\t{i}:", fmt_percent(to_fixes[i], to_count))

print("")

uk_count = len(f_modes[TraceFailure.FAST_UNKNOWN])
print("Fast Unknowns", uk_count)
print("fixed by")
for i in sorted(uk_fixes, key=lambda x: uk_fixes[x], reverse=True):
    print(f"\t{i}:", fmt_percent(uk_fixes[i], uk_count))

Timeouts 299
fixed by
	erase: 76.59%
	inst: 72.91%
	both: 49.83%

Fast Unknowns 66
fixed by
	inst: 96.97%
	erase: 31.82%
	both: 28.79%


In [8]:
import multiprocessing
from utils.cache_utils import *

def get_best_rank(rev: Evaluator):
    report = rev.build_report()
    mi = rev.get_trace_info()
    g = mi.get_trace_graph()
    ratios = rev.get_trace_graph_ratios()
    scores = dict()

    for qname, ratio in ratios.items():
        scores[qname] = g.aggregate_scores(ratio)

    scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    ranked = [x[0] for x in scores]
    best_rank = len(ranked)

    for qname in report.stabilized.qname.values:
        best_rank = min(best_rank, ranked.index(qname) + 1)

    return best_rank

def get_best_ranks(mode: TraceFailure):
    jobs = []

    for query in f_modes[mode]:
        rev = revs[query]
        # print(name_hash)
        ratio_cache = rev.name_hash + ".ratios"

        if not has_cache(ratio_cache):
            print("./src/debugger3.py -i", query, "--build-ratios")
            continue

        jobs.append(rev)

    pool = multiprocessing.Pool(16)
    return pool.map(get_best_rank, jobs)

summaries = dict()

for mode in [TraceFailure.SLOW_UNKNOWN, TraceFailure.NOT_FAIL, TraceFailure.TIMEOUT, TraceFailure.FAST_UNKNOWN]:
    summaries[mode] = get_best_ranks(mode)


./src/debugger3.py -i data/projs/bench_unstable/base.z3/fs_dice--queries-ASN1.Spec.Tag-2.smt2 --build-ratios
./src/debugger3.py -i data/projs/bench_unstable/base.z3/d_fvbkv--lib-Buckets-BucketsLib.i.dfy.Impl__BucketsLib.__default.SplitBucketOnPivotsAt.smt2 --build-ratios
./src/debugger3.py -i data/projs/bench_unstable/base.z3/d_fvbkv--lib-Buckets-BucketImpl.i.dfy.Impl__BucketImpl.MutBucket.WFBucketAt.smt2 --build-ratios
./src/debugger3.py -i data/projs/bench_unstable/base.z3/d_lvbkv--lib-Lang-System-PackedInts.s.dfy.CheckWellformed__NativePackedInts.__default.unpack__LittleEndian__Uint32__Seq.smt2 --build-ratios
./src/debugger3.py -i data/projs/bench_unstable/base.z3/d_fvbkv--lib-Marshalling-GenericMarshalling.i.dfy.Impl__GenericMarshalling.__default.MarshallCase_split0.smt2 --build-ratios
./src/debugger3.py -i data/projs/bench_unstable/base.z3/d_lvbkv--MapSpec-TSJ.i.dfy.Impl__TSJ.__default.Move1to2StepPreservesInv.smt2 --build-ratios
./src/debugger3.py -i data/projs/bench_unstable/bas

In [10]:
os.chdir("/home/amarshah/mariposa/src")
from calculate_average_rank import *
os.chdir("/home/yizhou7/mariposa/")


def get_name_hash(filename):
    import hashlib
    return hashlib.sha256(filename.encode()).hexdigest()[0:10]

fast_unknown_files = [get_name_hash(f) for f in f_modes[TraceFailure.FAST_UNKNOWN]]
timeout_files = files = [get_name_hash(f) for f in f_modes[TraceFailure.TIMEOUT]]
slow_unknown_files = files = [get_name_hash(f) for f in f_modes[TraceFailure.SLOW_UNKNOWN]]
not_fail_files = files = [get_name_hash(f) for f in f_modes[TraceFailure.NOT_FAIL]]

print(fast_unknown_files)




kw_parameters = {"ranking_heuristic": "proof_inst_graph"}
print(files)
fast_unknown_results = calculate_rankings(kw_parameters, fast_unknown_files)




['83448eecf5', '4ae002e825', 'cbc1128701', 'aa849323ae', 'f9f9100cfb', '712b931174', 'ed978bf127', 'a878753336', '2b4b7cd571', '4aecdc3251', '14aaa649c7', '80d7543acb', 'fa548b2af5', '40d72d4071', '6f64b72bc8', 'b778746723', 'a70ef5a023', 'acf6656f68', '873cd88973', 'fa1767e6e3', '1db53f149e', '4680269ef1', '28672c8a4e', 'b5cc583f2d', 'e6d6347750', '3818823b0a', '0a4b4299d2', 'fdbf242cbe', 'c2671f2f38', 'c8f75a1321', '2a2073ceb5', 'e05c339447', 'd87c685738', '5954cb98c8', '5ab30afc32', 'f8e63dc317', 'd8c158b8bf', '125461812b', 'c48bc62edc', '167389a8ef', 'cd6cdb377d', 'fcb903eff0', '1aa46bd795', '81df0e63de', 'e3b4e5b780', '79fffdb8cb', '842038cbbb', '9735a6b051', '7d2782e41a', '16d92ddc7b', 'dea7cbc5c5', '43588efb1c', '4bb10dc4e9', 'd8c62df78c', '7fedd43bf6', '2f7324b043', '519fdc89c1', 'e95adeebaf', '8852397c03', 'c8ca6d3f20', 'dd8e6c2233', '416446ca74', '462c420128', '3d63ac7eaf', 'de58a38d6c', '383f10cfb6']
['1a397d663b', '0b497bc090', '0334fcf203', 'a19f4272bf', 'fb0072addd', '473

[DEBUG] [edit] proof path: dbg/83448eecf5/proofs/reseed.444822085354912444.proof 
[DEBUG] [edit] trace path: dbg/83448eecf5/traces/rename.2780314412708427940 


[WARN] [differ] qid constructor_accessor_axiom not found in dbg/83448eecf5/orig.smt2 
cache/4ae002e825.report
b62705d7ff15b8bcf7969cb65dec214a.pickle


[DEBUG] [edit] proof path: dbg/4ae002e825/proofs/rename.8639222967174887047.proof 
[DEBUG] [edit] trace path: dbg/4ae002e825/traces/rename.525307100499888229 


[WARN] [differ] qid constructor_accessor_axiom not found in dbg/4ae002e825/orig.smt2 
cache/cbc1128701.report
9d742036e8cb3ce173c6e4358a4d9946.pickle


...

[DEBUG] [edit] proof path: dbg/de58a38d6c/proofs/shuffle.12295561062191406147.proof 
[DEBUG] [edit] trace path: dbg/de58a38d6c/traces/shuffle.8847520876316520251 


[WARN] [differ] qid constructor_accessor_axiom not found in dbg/de58a38d6c/orig.smt2 
cache/383f10cfb6.report
7c286244af1f6d1d3eed306db110a419.pickle


[DEBUG] [edit] proof path: dbg/383f10cfb6/proofs/shuffle.17774623702512382704.proof 
[DEBUG] [edit] trace path: dbg/383f10cfb6/traces/shuffle.14563076529949232051 


[WARN] [differ] qid constructor_accessor_axiom not found in dbg/383f10cfb6/orig.smt2 
[WARN] [differ] qid <null> not found in dbg/383f10cfb6/orig.smt2 
[1, 8, 3, 4, 65, 6, 16, 26, 1, 9, 9, 6, 4, 52, 1, 4, 4, 5, 3, 2, 5, 2, 3, 9, 2, 1, 2, 4, 2, 22, 2, 1, 3, 3, 8, 1, 5, 3, 1, 1, 32, 3, 3, 33, 2, 1, 1, 1, 6, 7, 40, 3, 15, 73, 1, 13, 48, 1, 63, 2, 15, 5, 10, 1, 2, 91]
Mean:  11
Median:  4.0
14 would fix on first try
32 would fix in 3 or fewer tries
51 would fix in 10 or fewer tries


[1,
 8,
 3,
 4,
 65,
 6,
 16,
 26,
 1,
 9,
 9,
 6,
 4,
 52,
 1,
 4,
 4,
 5,
 3,
 2,
 5,
 2,
 3,
 9,
 2,
 1,
 2,
 4,
 2,
 22,
 2,
 1,
 3,
 3,
 8,
 1,
 5,
 3,
 1,
 1,
 32,
 3,
 3,
 33,
 2,
 1,
 1,
 1,
 6,
 7,
 40,
 3,
 15,
 73,
 1,
 13,
 48,
 1,
 63,
 2,
 15,
 5,
 10,
 1,
 2,
 91]

In [11]:
kw_parameters = {"ranking_heuristic": "proof_inst_graph"}
print(files)
timeout_results = calculate_rankings(kw_parameters, timeout_files)

['1a397d663b', '0b497bc090', '0334fcf203', 'a19f4272bf', 'fb0072addd', '473fa6f229', '95ece4e2c8', '5b7d18ab3e']
cache/19e7b4a644.report
f7274bc4b0284f49b08fafb3c2c5702c.pickle


[DEBUG] [edit] proof path: dbg/19e7b4a644/proofs/shuffle.17669631069866628208.proof 
[DEBUG] [edit] trace path: dbg/19e7b4a644/traces/reseed.6283199569595562931 


[WARN] [differ] qid constructor_accessor_axiom not found in dbg/19e7b4a644/orig.smt2 
cache/802c68cc26.report
5780f8e1c89a9afd3eef90a905220b1e.pickle


[DEBUG] [edit] proof path: dbg/802c68cc26/proofs/reseed.18003067991515608241.proof 
[DEBUG] [edit] trace path: dbg/802c68cc26/traces/reseed.2674813137586048871 


[WARN] [differ] qid constructor_accessor_axiom not found in dbg/802c68cc26/orig.smt2 
[WARN] [differ] qid <null> not found in dbg/802c68cc26/orig.smt2 
cache/e0ef4bae19.report
e687b0f08cd8a5fc08f69b3e5bbcbfe6.pickle


...

[DEBUG] [edit] proof path: dbg/c174525f75/proofs/rename.15748788026567450328.proof 
[DEBUG] [edit] trace path: dbg/c174525f75/traces/rename.5850373615816341614 


[WARN] [differ] qid constructor_accessor_axiom not found in dbg/c174525f75/orig.smt2 
[WARN] [differ] qid <null> not found in dbg/c174525f75/orig.smt2 
cache/53b94c898b.report
dd5ae8bd0d3cd13ca5cc8df36dcc2ef8.pickle


[DEBUG] [edit] proof path: dbg/53b94c898b/proofs/rename.3761459269803675162.proof 
[DEBUG] [edit] trace path: dbg/53b94c898b/traces/shuffle.2911831914072920678 


[WARN] [differ] qid constructor_accessor_axiom not found in dbg/53b94c898b/orig.smt2 
[WARN] [differ] qid <null> not found in dbg/53b94c898b/orig.smt2 
cache/d27df8e3bb.report
6b2a86130dcd1b2ac60674c7a23c3da9.pickle


[DEBUG] [edit] proof path: dbg/d27df8e3bb/proofs/reseed.3098858813203874436.proof 
[DEBUG] [edit] trace path: dbg/d27df8e3bb/traces/rename.7821953825153783066 
Exception ignored in: <function Z3AstVisitor.__del__ at 0x7febbe36a700>
Traceback (most recent call last):
  File "/home/yizhou7/mariposa/src/debugger/z3_utils.py", line 200, in __del__
    def __del__(self):
KeyboardInterrupt: 


kw_parameters = {"ranking_heuristic": "proof_inst_graph"}
print(files)
calculate_rankings(kw_parameters, fast_unknown_files)

In [16]:
os.chdir("/home/amarshah/mariposa/src")
from calculate_average_rank import *
os.chdir("/home/yizhou7/mariposa/")

slow_unknown_files = files = [get_name_hash(f) for f in f_modes[TraceFailure.SLOW_UNKNOWN]]
not_fail_files = files = [get_name_hash(f) for f in f_modes[TraceFailure.NOT_FAIL]]

kw_parameters = {"ranking_heuristic": "proof_inst_graph"}
slow_unknown_results = calculate_rankings(kw_parameters, slow_unknown_files)

cache/3ce5c5e4dc.report
281fbb5d0d8857c429d29610649306bf.pickle


[DEBUG] [edit] proof path: dbg/3ce5c5e4dc/proofs/reseed.5498765500434424855.proof 
[DEBUG] [edit] trace path: dbg/3ce5c5e4dc/traces/reseed.5797641433228418118 


[WARN] [differ] qid constructor_accessor_axiom not found in dbg/3ce5c5e4dc/orig.smt2 
[WARN] [differ] qid unnamed not found in dbg/3ce5c5e4dc/orig.smt2 
[WARN] [differ] qid is-Prims.equals not found in dbg/3ce5c5e4dc/orig.smt2 
[WARN] [differ] qid is-Prims.h_equals not found in dbg/3ce5c5e4dc/orig.smt2 
[WARN] [differ] qid is-Prims.c_and not found in dbg/3ce5c5e4dc/orig.smt2 
[WARN] [differ] qid is-Prims.c_or not found in dbg/3ce5c5e4dc/orig.smt2 
[WARN] [differ] qid is-Prims.dtuple2 not found in dbg/3ce5c5e4dc/orig.smt2 
[WARN] [differ] qid is-Prims.list not found in dbg/3ce5c5e4dc/orig.smt2 
[WARN] [differ] qid is-FStar.Pervasives.Native.option not found in dbg/3ce5c5e4dc/orig.smt2 
[WARN] [differ] qid is-FStar.Pervasives.Native.tuple2 not found in dbg/3ce5c5e4dc/orig.smt2 
[WARN] [differ] qid is-FStar.Pervasives.Native.tuple3 not found in dbg/3ce5c5e4dc/orig.smt2 
[WARN] [differ] qid is-FStar.Pervasives.Native.tuple4 not found in dbg/3ce5c5e4dc/orig.smt2 
[WARN] [differ] qid is-FSta

[DEBUG] [edit] proof path: dbg/e998d08b31/proofs/reseed.10397248321928259864.proof 
[DEBUG] [edit] trace path: dbg/e998d08b31/traces/reseed.5819383338623026635 


[WARN] [differ] qid constructor_accessor_axiom not found in dbg/e998d08b31/orig.smt2 
[WARN] [differ] qid <null> not found in dbg/e998d08b31/orig.smt2 
[2, 8]
Mean:  5
Median:  5.0
0 would fix on first try
1 would fix in 3 or fewer tries
2 would fix in 10 or fewer tries


In [20]:
kw_parameters = {"ranking_heuristic": "proof_inst_graph"}
print(files)
not_fail_results = calculate_rankings(kw_parameters, not_fail_files)

['95ece4e2c8', '1a397d663b', 'fb0072addd', '0b497bc090', '473fa6f229', 'a19f4272bf', '5b7d18ab3e', '0334fcf203']
cache/95ece4e2c8.report
0fd01710237478933ae5dc9fb99eb436.pickle


[DEBUG] [edit] proof path: dbg/95ece4e2c8/proofs/shuffle.6507208572592150382.proof 
[DEBUG] [edit] trace path: dbg/95ece4e2c8/traces/rename.16904878405275336679 


[WARN] [differ] qid constructor_accessor_axiom not found in dbg/95ece4e2c8/orig.smt2 
[WARN] [differ] qid <null> not found in dbg/95ece4e2c8/orig.smt2 
cache/1a397d663b.report
3d64aa7ecaf6b8164a75e3fe0dfbc8e9.pickle


[DEBUG] [edit] proof path: dbg/1a397d663b/proofs/shuffle.9209495299167861333.proof 
[DEBUG] [edit] trace path: dbg/1a397d663b/traces/rename.3384331697473390472 


[WARN] [differ] qid constructor_accessor_axiom not found in dbg/1a397d663b/orig.smt2 
cache/fb0072addd.report
e9d40ef80d43527a5f677faa3c8bc81e.pickle


...

[WARN] [differ] qid constructor_accessor_axiom not found in dbg/a19f4272bf/orig.smt2 
[WARN] [differ] qid <null> not found in dbg/a19f4272bf/orig.smt2 
cache/5b7d18ab3e.report
277708ee08844f5c5ae559f5370a0fbc.pickle


[DEBUG] [edit] proof path: dbg/5b7d18ab3e/proofs/shuffle.4004227050859073429.proof 
[DEBUG] [edit] trace path: dbg/5b7d18ab3e/traces/rename.16113115495686912387 


[WARN] [differ] qid constructor_accessor_axiom not found in dbg/5b7d18ab3e/orig.smt2 
cache/0334fcf203.report
d98c6b657ba36fb0f034c09793e5b0b4.pickle


[DEBUG] [edit] proof path: dbg/0334fcf203/proofs/shuffle.3767154315890758019.proof 
[DEBUG] [edit] trace path: dbg/0334fcf203/traces/reseed.5061766174274051476 


[WARN] [differ] qid constructor_accessor_axiom not found in dbg/0334fcf203/orig.smt2 
[1, 1, 5, 1, 2, 9, 1, 18]
Mean:  4
Median:  1.5
4 would fix on first try
5 would fix in 3 or fewer tries
7 would fix in 10 or fewer tries


In [10]:
import numpy as np

for mode in [TraceFailure.TIMEOUT, TraceFailure.FAST_UNKNOWN, TraceFailure.SLOW_UNKNOWN, TraceFailure.NOT_FAIL]:
    best_ranks = summaries[mode]
    ranks = np.array(best_ranks)
    print(mode, len(best_ranks))
    print(np.where(ranks == 1)[0].shape[0], "would fix on first try")
    print(np.where(ranks <= 3)[0].shape[0], "would fix in 3 or fewer tries")
    print(np.where(ranks <= 10)[0].shape[0], "would fix in 10 or fewer tries")
    print("")






TraceFailure.TIMEOUT 299
114 would fix on first try
173 would fix in 3 or fewer tries
238 would fix in 10 or fewer tries

TraceFailure.FAST_UNKNOWN 66
9 would fix on first try
18 would fix in 3 or fewer tries
35 would fix in 10 or fewer tries

TraceFailure.SLOW_UNKNOWN 0
0 would fix on first try
0 would fix in 3 or fewer tries
0 would fix in 10 or fewer tries

TraceFailure.NOT_FAIL 0
0 would fix on first try
0 would fix in 3 or fewer tries
0 would fix in 10 or fewer tries



In [23]:
timeout_ranks = summaries[TraceFailure.TIMEOUT]
total_ranks = timeout_ranks + fast_unknown_results + slow_unknown_results + not_fail_results
ranks = np.array(total_ranks)

print("CUMULATIVE", len(total_ranks))
print("mean: ", mean(total_ranks))
print("median: ", median(total_ranks))
print(np.where(ranks == 1)[0].shape[0], "would fix on first try")
print(np.where(ranks <= 3)[0].shape[0], "would fix in 3 or fewer tries")
print(np.where(ranks <= 10)[0].shape[0], "would fix in 10 or fewer tries")
print("")

CUMULATIVE 375
mean:  16
median:  3
132 would fix on first try
211 would fix in 3 or fewer tries
298 would fix in 10 or fewer tries



In [5]:
revs = BenchViewer({'data/projs/anvil/base.z3/fluent-smt-fluent_controller__fluentbit__proof__liveness__resource_match__lemma_from_after_get_resource_step_to_after_update_resource_step.smt2', 'data/projs/anvil/base.z3/fluent-smt-fluent_controller__fluentbit__proof__helper_invariants__validation.3.smt2', 'data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__setup_v.5.smt2', 'data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__inv_v.11.smt2', 'data/projs/vsystemsnew/base.z3/mimalloc-smt-page_organization__PageOrg.69.smt2', 'data/projs/verismo.dedup/base.z3/snp__ghcb__proto_page.141.smt2', 'data/projs/anvil/base.z3/zookeeper-smt-zookeeper_controller__proof__liveness__stateful_set_match.2.smt2', 'data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-log__inv_v.13.smt2', 'data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__inv_v.16.smt2', 'data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__inv_v.15.smt2', 'data/projs/verismo.dedup/base.z3/bsp__ap.3.smt2', 'data/projs/verismo.dedup/base.z3/vbox__vbox.4.smt2', 'data/projs/anvil/base.z3/zookeeper-smt-zookeeper_controller__proof__helper_invariants__proof__lemma_eventually_always_no_delete_resource_request_msg_in_flight.smt2', 'data/projs/anvil/base.z3/zookeeper-smt-rabbitmq-smt-rabbitmq_controller__proof__helper_invariants__validation__lemma_always_stateful_set_in_create_request_msg_satisfies_unchangeable.smt2', 'data/projs/splinterdb/base.z3/allocation_layer__LikesBetree_v.18.smt2', 'data/projs/anvil/base.z3/zookeeper-smt-rabbitmq-smt-rabbitmq_controller__proof__liveness__stateful_set_match__lemma_from_after_get_stateful_set_step_to_after_update_stateful_set_step.smt2', 'data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-log__logimpl_v.6.smt2', 'data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__inv_v.2.smt2', 'data/projs/splinterdb/base.z3/allocation_layer__LikesBetree_v__LikesBetree.3.smt2', 'data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-log__inv_v.2.smt2', 'data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__inv_v.9.smt2', 'data/projs/verismo.dedup/base.z3/boot__init__e820_init.4.smt2', 'data/projs/anvil/base.z3/zookeeper-smt-rabbitmq-smt-rabbitmq_controller__proof__helper_invariants__proof__lemma_eventually_always_every_resource_create_request_implies_at_after_create_resource_step.smt2', 'data/projs/anvil/base.z3/fluent-smt-fluent_controller__fluentbit__proof__liveness__daemon_set_match__lemma_from_after_get_daemon_set_step_to_after_update_daemon_set_step.smt2', 'data/projs/anvil/base.z3/zookeeper-smt-zookeeper_controller__proof__helper_invariants__validation.3.smt2', 'data/projs/vsystemsnew/base.z3/mimalloc-smt-segment__segment_span_free_coalesce_before.smt2', 'data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-log__layout_v.30.smt2', 'data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__inv_v.14.smt2', 'data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__inv_v.13.smt2', 'data/projs/anvil/base.z3/fluent-smt-fluent_controller__fluentbit_config__proof__liveness__resource_match.5.smt2', 'data/projs/anvil/base.z3/zookeeper-smt-rabbitmq-smt-rabbitmq_controller__proof__liveness__resource_match.1.smt2', 'data/projs/vsystemsnew/base.z3/mimalloc-smt-commit_segment.1.smt2', 'data/projs/verismo.dedup/base.z3/snp__cpuid.17.smt2', 'data/projs/anvil/base.z3/fluent-smt-fluent_controller__fluentbit_config__proof__liveness__resource_match.6.smt2', 'data/projs/verismo.dedup/base.z3/vbox__vbox.5.smt2', 'data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-log__inv_v.17.smt2', 'data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__inv_v.17.smt2', 'data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__inv_v.7.smt2', 'data/projs/anvil/base.z3/zookeeper-smt-zookeeper_controller__proof__helper_invariants__proof.2.smt2', 'data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-log__inv_v.10.smt2', 'data/projs/anvil/base.z3/zookeeper-smt-rabbitmq-smt-rabbitmq_controller__proof__liveness__stateful_set_match__lemma_stateful_set_state_matches_at_after_update_stateful_set_step.smt2', 'data/projs/vsystemsnew/base.z3/page-table-smt-impl_u__l2_refinement.3.smt2', 'data/projs/vsystemsnew/base.z3/mimalloc-smt-config.35.smt2', 'data/projs/verismo.dedup/base.z3/security__monitor.21.smt2', 'data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-log__inv_v.4.smt2', 'data/projs/verismo.dedup/base.z3/boot__init__e820_init_alloc.1.smt2', 'data/projs/verismo.dedup/base.z3/boot__init__e820_init.2.smt2', 'data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-log__inv_v.12.smt2', 'data/projs/vsystemsnew/base.z3/mimalloc-smt-segment__segment_os_alloc.smt2', 'data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-log__inv_v.5.smt2', 'data/projs/anvil/base.z3/fluent-smt-fluent_controller__fluentbit__proof__helper_invariants__proof.9.smt2', 'data/projs/verismo.dedup/base.z3/pgtable_e__pte.20.smt2', 'data/projs/anvil/base.z3/fluent-smt-fluent_controller__fluentbit_config__proof__liveness__resource_match__lemma_from_after_get_resource_step_to_after_create_resource_step.smt2', 'data/projs/anvil/base.z3/zookeeper-smt-zookeeper_controller__proof__helper_invariants__zookeeper_api__lemma_zk_request_implies_step_helper.smt2', 'data/projs/anvil/base.z3/zookeeper-smt-zookeeper_controller__proof__helper_invariants__proof__lemma_always_no_update_status_request_msg_in_flight_of_except_stateful_set.smt2', 'data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__start_v.2.smt2', 'data/projs/verismo.dedup/base.z3/pgtable_e__pte.18.smt2', 'data/projs/verismo.dedup/base.z3/allocator__linkedlist.10.smt2', 'data/projs/atmosphere/base.z3/kernel__create_and_share_pages.5.smt2', 'data/projs/anvil/base.z3/zookeeper-smt-rabbitmq-smt-rabbitmq_controller__proof__helper_invariants__proof__lemma_eventually_always_object_in_response_at_after_create_resource_step_is_same_as_etcd.smt2', 'data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__setup_v.6.smt2', 'data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-log__inv_v.14.smt2', 'data/projs/vsystemsnew/base.z3/mimalloc-smt-segment__span_queue_delete.smt2'})

[WARN] [eval] 82449bd2e8 tested count 112 < registered 338 
[WARN] [eval] 66c9bdf69f tested count 1225 < registered 1612 
